In [1]:
import torch

In [30]:
torch.manual_seed(0)

d_k = 3
B = 2
N = 5
T = 6
n_heads = 4

Q = torch.randn((B, n_heads, N, d_k))
K = torch.randn((B, n_heads, N, d_k))
V = torch.randn((B, n_heads, N, d_k))

In [31]:
def attn(Q, K, V, mask):
	QKT = (Q@K.transpose(-2,-1)) * (d_k**-.5)
	masked_QKT = QKT.masked_fill(mask, float("-inf"))
	weights = torch.softmax(masked_QKT, dim=-1)
	return weights@V

attn(Q, K, V, mask=torch.triu(torch.ones((N, N), dtype=torch.bool), diagonal=1)).shape

torch.Size([2, 4, 5, 3])

Just to check that the @ operator does a batched matrix multiply how I like let's unroll and use BMM

In [32]:
torch.manual_seed(0)
a = torch.randn((B, n_heads, N, d_k))
b = torch.randn((B, n_heads, N, d_k))

In [33]:
a_unrolled = a.view((-1, N, d_k))
b_unrolled = b.view((-1, N, d_k))
bmm = torch.bmm(a_unrolled, b_unrolled.transpose(-2, -1)).view((B, n_heads, N, N)).contiguous()
regular = a@b.transpose(-2, -1)
torch.allclose(bmm, regular)

True